
# Taller 1 - Fase 3: Aplicación de la Ingeniería de Prompts (EcoMarket)

Este notebook desarrolla la **Fase 3 del Taller IAGEN**, aplicando **ingeniería de prompts**
para el caso de estudio **EcoMarket** (optimización del servicio al cliente en e-commerce).  

Se usará un **dataset simulado de pedidos (`pedidos.xlsx`)** y la **API de OpenAI** para probar:
- Consultas de **estado de pedido**  
- Consultas de **devolución de producto**  


In [ ]:

!pip install openai pandas


In [ ]:

import pandas as pd
from openai import OpenAI

# Cargar dataset de pedidos simulado
df = pd.read_excel("/content/pedidos.xlsx")
df


,tracking_number,producto,estado,fecha_entrega_estimada,link_rastreo,comentario
0,12345,Botella Reusable,En camino,2025-10-01,www.eco.com/track/12345,En tránsito
1,23456,Shampoo sólido,Retrasado,2025-10-05,www.eco.com/track/23456,Retraso por aduanas
2,34567,Cepillo de bambú,Entregado,2025-09-26,www.eco.com/track/34567,Entregado al cliente
3,45678,Detergente ecológico,En preparación,2025-09-29,www.eco.com/track/45678,Pendiente de despacho
4,56789,Jabón artesanal,Cancelado,2025-09-30,www.eco.com/track/56789,Pedido cancelado por cliente


In [ ]:

system_prompt = """
Eres un asesor experto en atención al cliente de EcoMarket.
Debes responder con empatía, claridad y precisión.
Consulta siempre la base de datos interna de pedidos o políticas de devoluciones.
"""

def build_prompt_estado_pedido(tracking_number, pedido):
    return f"""
Actúa como un agente de atención al cliente de EcoMarket.
El cliente solicita el estado del pedido con número de seguimiento {tracking_number}.

Información encontrada en la base de datos:
- Estado: {pedido['estado']}
- Fecha estimada de entrega: {pedido['fecha_entrega_estimada']}
- Enlace de rastreo: {pedido['link_rastreo']}
- Comentario: {pedido['comentario']}

Genera una respuesta amable y clara para el cliente.
Si el pedido está retrasado, pide disculpas y explica el motivo.
"""

def build_prompt_devolucion(producto, elegible):
    if elegible:
        return f"""
El cliente desea devolver el producto '{producto}'.
Este producto **sí es elegible** para devolución según las políticas de EcoMarket.

Guía al cliente con pasos claros para iniciar la devolución (plazos, pasos y condiciones).
Responde siempre con empatía y claridad.
"""
    else:
        return f"""
El cliente desea devolver el producto '{producto}'.
Este producto **NO es elegible** para devolución (ej. productos de higiene o perecederos).

Responde con empatía y explica claramente la razón, ofreciendo alternativas si existen.
"""


In [ ]:

# Configura tu API Key antes de usar
client = OpenAI(api_key="sk-API_KEY")

def consultar_estado_pedido(tracking_number):
    pedido = df[df["tracking_number"] == tracking_number]
    if pedido.empty:
        return f"No se encontró el pedido con número {tracking_number}."
    pedido_info = pedido.iloc[0]
    user_prompt = build_prompt_estado_pedido(tracking_number, pedido_info)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

def procesar_devolucion(producto):
    # Definir criterios de elegibilidad
    no_elegibles = ["Shampoo sólido", "Jabón artesanal"]
    elegible = producto not in no_elegibles

    user_prompt = build_prompt_devolucion(producto, elegible)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content


In [ ]:

# Ejemplo: Estado de pedido
print(consultar_estado_pedido(23456))

# Ejemplo: Devolución de producto
print(procesar_devolucion("Cepillo de bambú"))
print(procesar_devolucion("Shampoo sólido"))


Estimado/a cliente,

Gracias por contactarnos y por su paciencia. He revisado el estado de su pedido con el número de seguimiento 23456 y lamentablemente, debo informarle que está en un estado de retraso.

La nueva fecha estimada de entrega es para el 5 de octubre de 2025. El motivo de este retraso se debe a un inconveniente con las aduanas. Entendemos que esto puede ser frustrante y queremos disculparnos sinceramente por cualquier inconveniente que esto le pueda causar.

Para que pueda realizar un seguimiento de su pedido, aquí le dejo el enlace donde podrá ver su progreso: [www.eco.com/track/23456](www.eco.com/track/23456).

Si tiene alguna otra pregunta o necesita más asistencia, no dude en hacérmelo saber. Estoy aquí para ayudarle.

Saludos cordiales,  
[Su nombre]  
EcoMarket Atención al Cliente
¡Hola! Muchas gracias por ponerte en contacto con nosotros. Entendemos que deseas devolver el 'Cepillo de bambú', y estaré encantado de guiarte en el proceso.

Aquí te detallo los pasos a 